In [1]:
import torch.nn as nn 
from DataLoader import DataLoader 
import torch.optim as optim
from Models import RotationNN
from utils import plot_image, line_image, rotate_images

In [307]:
image_size = 512
num_samples = 100
batch_size = 1 # pure SGD

loader = DataLoader(image_size, num_samples, batch_size)
model = RotationNN(image_size)
criterion = nn.BCELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.05) 

In [ ]:
epochs = 100
for epoch in range(epochs):
    for x, _ in loader:
        y = rotate_images(x)
        y_pred = model(x)
    
        loss = criterion(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')